In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import os,random, math, psutil, pickle

import pandas_profiling as pdp

from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

import category_encoders as ce


In [2]:
train = pd.read_csv('./input/bank_marketing_train.csv')

In [3]:
train.shape

(33744, 20)

In [4]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [5]:
train.head()

,age,job,marital,education,default,housing,loan,contact,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,56,services,married,high.school,no,no,yes,telephone,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,45,services,married,basic.9y,unknown,no,no,telephone,mon,198,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,59,admin.,married,professional.course,no,no,no,telephone,mon,139,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [6]:
train.job.value_counts()

admin.           8536
blue-collar      7591
technician       5529
services         3255
management       2359
retired          1432
entrepreneur     1203
self-employed    1130
housemaid         882
unemployed        822
student           735
unknown           270
Name: job, dtype: int64

In [7]:
train["nr.employed"].value_counts()

5228.1    13276
5099.1     6987
5191.0     6389
5195.8     3014
5076.2     1362
5017.5      880
4991.6      633
5008.7      528
4963.6      524
5023.5      143
5176.3        8
Name: nr.employed, dtype: int64

In [8]:
# pdp.ProfileReport(train)

In [9]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y'],
      dtype='object')

# 前処理

In [10]:
# durationをdropする
numeric = [ 'cons.price.idx', 'cons.conf.idx', 'age','campaign','pdays', 'previous'  ]
categoricals = ['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week',  
       'poutcome', 'emp.var.rate', 
       'euribor3m', 'nr.employed']

feat_cols = numeric + categoricals


In [11]:
def preprocessing ( df ) :
    df.drop(['duration'], axis=1, inplace=True)
    df["y"].replace("no",0, inplace=True)
    df["y"].replace("yes",1, inplace=True)
    ce_oe = ce.OrdinalEncoder(cols=categoricals,handle_unknown='impute')
    df = ce_oe.fit_transform(df)

    #oe = preprocessing.OrdinalEncoder()
#     encoded = oe.fit_transform(df[categoricals].values)
#     le = LabelEncoder()
#     df[[categoricals]] = le.fit_transform(df[[categoricals]])
    return df


In [12]:
train = preprocessing(train)

In [13]:
train

,age,job,marital,education,default,housing,loan,contact,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,1,1,1,1,1,1,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
1,57,2,1,2,2,1,1,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
2,56,2,1,2,1,1,2,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
3,45,2,1,3,2,1,1,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
4,59,3,1,4,1,1,1,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
5,41,4,1,5,2,1,1,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
6,24,5,2,4,1,2,1,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
7,25,2,2,2,1,2,1,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
8,29,4,2,2,1,1,2,1,1,1,999,0,1,1,93.994,-36.4,1,1,0
9,57,1,3,1,1,2,1,1,1,1,999,0,1,1,93.994,-36.4,1,1,0


# モデル作成

In [14]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y'],
      dtype='object')

In [15]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'rmse'},
            'subsample': 0.7,
            'subsample_freq': 1,
            'learning_rate': 0.03,
            'num_leaves': 30,
            'feature_fraction': 0.8,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'min_data_in_leaf': 30,
            #'label_gain':[1,10]
            'scale_pos_weight':9
            #'device':'gpu'
            }

# folds = 5
# seed = 666
# kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

folds = 5
seed = 666

kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
#kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

In [16]:
#(train_index, val_index) = kf.split(train, train["y"])
# hoge = kf.split(train, train["y"])

i =0
for train_index, val_index in kf.split(train, train['y']):
    train_X = train[feat_cols].iloc[train_index] 
    train_y = train["y"].iloc[train_index] 
    val_X = train[feat_cols].iloc[val_index]
    val_y = train["y"].iloc[val_index]
    i = i+1
    print(1)
    break



1


In [17]:
train_X

#train_X_new = pd.DataFrame(train_X, index=train_X_origin.index , columns=train_X_origin.columns)

,cons.price.idx,cons.conf.idx,age,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,day_of_week,poutcome,emp.var.rate,euribor3m,nr.employed
0,93.994,-36.4,56,1,999,0,1,1,1,1,1,1,1,1,1,1,1,1
1,93.994,-36.4,57,1,999,0,2,1,2,2,1,1,1,1,1,1,1,1
2,93.994,-36.4,56,1,999,0,2,1,2,1,1,2,1,1,1,1,1,1
3,93.994,-36.4,45,1,999,0,2,1,3,2,1,1,1,1,1,1,1,1
5,93.994,-36.4,41,1,999,0,4,1,5,2,1,1,1,1,1,1,1,1
6,93.994,-36.4,24,1,999,0,5,2,4,1,2,1,1,1,1,1,1,1
7,93.994,-36.4,25,1,999,0,2,2,2,1,2,1,1,1,1,1,1,1
8,93.994,-36.4,29,1,999,0,4,2,2,1,1,2,1,1,1,1,1,1
9,93.994,-36.4,57,1,999,0,1,3,1,1,2,1,1,1,1,1,1,1
10,93.994,-36.4,35,1,999,0,4,1,6,1,2,1,1,1,1,1,1,1


In [18]:
from imblearn.over_sampling import SMOTE

train_X_origin = train_X.copy()
train_y_origin = train_y.copy()

sm = SMOTE(random_state=42)
train_X, train_y = sm.fit_sample(train_X, train_y)

p = list(zip(train_X, train_y))
random.shuffle(p)
train_X, train_y = zip(*p)


In [19]:
train_X = pd.DataFrame(train_X, columns=train_X_origin.columns)
train_y = np.array(train_y)

In [20]:
lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)




In [21]:
train_y.sum()
train_y.shape

pd.DataFrame(train_y).tail(10)

,0
47822,0
47823,0
47824,0
47825,0
47826,0
47827,1
47828,1
47829,0
47830,0
47831,0


In [22]:
exp_val = 0

In [23]:
def exp_val_func (y_pred , data):
    valid_y = data.get_label().tolist()
    tmp_exp_val = 0
#     print(valid_y)
    for i in range (len(valid_y)) :
        if y_pred[i]:
            tmp_exp_val = tmp_exp_val - 500
            if valid_y[i]:
                tmp_exp_val = tmp_exp_val + 3000

       # exp_val += tmp_exp_val
#         if i % 1000 == 999 :
#             print(tmp_exp_val)
    return 'exp_val', tmp_exp_val, True


In [24]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=2000,
                verbose_eval = 100)
#               feval = exp_val_func)

Training until validation scores don't improve for 2000 rounds.
[100]	training's rmse: 0.451789	valid_1's rmse: 0.595153
[200]	training's rmse: 0.410822	valid_1's rmse: 0.547417
[300]	training's rmse: 0.381375	valid_1's rmse: 0.514179
[400]	training's rmse: 0.361946	valid_1's rmse: 0.493554
[500]	training's rmse: 0.345698	valid_1's rmse: 0.477219
[600]	training's rmse: 0.333494	valid_1's rmse: 0.465532
[700]	training's rmse: 0.323305	valid_1's rmse: 0.45632
[800]	training's rmse: 0.314031	valid_1's rmse: 0.448616
[900]	training's rmse: 0.30504	valid_1's rmse: 0.440816
[1000]	training's rmse: 0.2975	valid_1's rmse: 0.435161
[1100]	training's rmse: 0.290387	valid_1's rmse: 0.429705
[1200]	training's rmse: 0.283342	valid_1's rmse: 0.424247
[1300]	training's rmse: 0.277755	valid_1's rmse: 0.42058
[1400]	training's rmse: 0.271829	valid_1's rmse: 0.416568
[1500]	training's rmse: 0.266243	valid_1's rmse: 0.412546
[1600]	training's rmse: 0.260655	valid_1's rmse: 0.408766
[1700]	training's rmse

[14100]	training's rmse: 0.0789226	valid_1's rmse: 0.339095
[14200]	training's rmse: 0.0784723	valid_1's rmse: 0.338976
[14300]	training's rmse: 0.078128	valid_1's rmse: 0.338972
[14400]	training's rmse: 0.0777399	valid_1's rmse: 0.338858
[14500]	training's rmse: 0.0773108	valid_1's rmse: 0.338851
[14600]	training's rmse: 0.0771253	valid_1's rmse: 0.339065
[14700]	training's rmse: 0.076655	valid_1's rmse: 0.338824
[14800]	training's rmse: 0.0764051	valid_1's rmse: 0.338891
[14900]	training's rmse: 0.0760216	valid_1's rmse: 0.338874
[15000]	training's rmse: 0.0757308	valid_1's rmse: 0.338906
[15100]	training's rmse: 0.0755099	valid_1's rmse: 0.338966
[15200]	training's rmse: 0.0750961	valid_1's rmse: 0.338816
[15300]	training's rmse: 0.0747932	valid_1's rmse: 0.338791
[15400]	training's rmse: 0.0744991	valid_1's rmse: 0.3388
[15500]	training's rmse: 0.0741335	valid_1's rmse: 0.338618
[15600]	training's rmse: 0.0737627	valid_1's rmse: 0.338634
[15700]	training's rmse: 0.0735265	valid_1's

In [25]:
fi = pd.DataFrame(  {0: train_X.columns ,  1: gbm.feature_importance()})
fi

,0,1
0,cons.price.idx,20339
1,cons.conf.idx,17338
2,age,113359
3,campaign,48222
4,pdays,4525
5,previous,6605
6,job,38929
7,marital,22823
8,education,27234
9,default,10909


In [26]:
from sklearn.metrics import confusion_matrix
# y_true = [0, 0, 0, 0, 1, 1, 1, 0, 1, 0]   # 実際の値 (0:スパムでない, 1:スパム)
# y_pred = [0, 0, 0, 0, 1, 1, 1, 1, 0, 1]   # 識別結果 (0:スパムでない, 1:スパム)
y_pred = gbm.predict(val_X)


In [27]:
y_pred.tolist()

[0.03199867444903173,
 0.031094666188295896,
 0.000417193809907825,
 0.0014304459120553577,
 0.0015118009489280039,
 0.13922228845422,
 0.022379184232547967,
 0.016461457227412594,
 0.0008940553779171581,
 0.015521010599622916,
 0.02430702385284745,
 0.3765112500664252,
 0.0010882121905908446,
 0.002212002505598792,
 0.0985213001776478,
 0.0005203697027013806,
 0.8982791634107866,
 0.12044252603817672,
 7.131127312537799e-06,
 0.0032301802952891335,
 5.319467696635738e-05,
 0.0019703144037133847,
 0.003515143065046285,
 0.002450131284372787,
 0.0015067342100441314,
 0.05755171966531615,
 0.0016095533836238995,
 0.0037782809803872464,
 0.0030353171895918457,
 0.016703976706535836,
 0.00029307571266153087,
 0.00015663148073430773,
 0.004370585826853544,
 0.020766188697601124,
 0.021636269809096262,
 0.14706333585546166,
 0.28663718800792504,
 0.0053306443258747775,
 0.0007554112902274383,
 0.00022529009803816448,
 0.061594136915284106,
 0.019238807123131245,
 0.25657107428015796,
 0.0048

In [28]:
val_y.tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [29]:
val_y_c = val_y.tolist()
y_pred_c =  y_pred.tolist()


In [30]:
labels = ["not cvr", "cvr"]
#confusion_matrix( val_y_c, y_pred.round(), labels=labels, index=labels)

from pycm import ConfusionMatrix
cm = ConfusionMatrix(actual_vector=val_y_c, predict_vector=y_pred.round(0).astype("u1") )  
cm.print_matrix()
#confusion_matrix( val_y_c, y_pred.round(), labels=labels, index=labels)

Predict    0          1          
Actual
0          5499       533        

1          400        317        


